In [1]:
pip install kafka-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.3/326.3 kB 1.3 MB/s eta 0:00:00a 0:00:01m

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.4/455.4 MB 6.9 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached py4j-0.10.9.9-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached py4j-0.10.9.9-py2.py3-none-any.whl (203 kB)
  Created wheel for pyspark: filename=pyspark-4.1.1-py2.py3-none-any.whl size=456008707 sha256=9c455427a4d68cea0ddba254410ed44201a0ba212f1c8370c724cb1e0a7f79c3
  Stored in directory: /Users/Rom/Library/Caches/pip/wheels/16/33/a9/f8bff354a182417214933df74dace2a34b02c3e5643e8fac74
Successfully built pyspark

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pyspark

In [4]:
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_2.13:{pyspark.__version__} pyspark-shell'
os.environ['SPARK_SUBMIT_OPTS'] = '-Djdk.security.auth.login.Config=ignore'


In [5]:
KAFKA_BROKER_URL = "localhost:9092"
KAFKA_TOPIC = "wikimedia_topic_1"

In [6]:
pip install requests


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install sseclient

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=5637 sha256=e6b8eb08409312765d2ce4c79a20107372ce588c3a953e5e983619ef92f449a7
  Stored in directory: /Users/Rom/Library/Caches/pip/wheels/7c/54/eb/a223b1599728ecaf0528281c17c96c503aa7d18a752a4e4e3a
Successfully built sseclient

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import threading
import time
from kafka import KafkaConsumer
from kafka import KafkaProducer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, expr
from pyspark.sql.types import StructType, StringType, IntegerType
from pyspark.streaming import StreamingContext


In [9]:
# *** Note!!! **** At this point you need to have Kafka broker running. See Setup for Docker and Kafka.

producer = KafkaProducer(bootstrap_servers=KAFKA_BROKER_URL)

In [10]:
# *** Note!!! **** In order to read wikimedia changes, you need to obtain Access Token 
# See instructions in Readm about how to authenticate to Wikimedia page

In [11]:
import requests
from sseclient import SSEClient
URL = 'https://stream.wikimedia.org/v2/stream/recentchange'
headers = {
    "User-Agent": "[YOUR_APP_NAME_HERE]",
    "Authorization": "Bearer [YOUR_ACCESS_TOKEN_HERE]"
}
def relay():
    events = SSEClient(URL, headers=headers, timeout=30)
    for i in range(100):
        for event in events:
            if event.event == 'message' and event.data != None:
                message = event.data.encode("utf-8")
                producer.send(KAFKA_TOPIC, value=message)
                break
threading.Thread(target=relay).start()


In [12]:
# *** Note!!! **** Before continue to the next phase, make sure that you have a topic and events in it.
# See 'Lookup Kafka Topics' in the Readme.

In [13]:
spark = SparkSession.builder \
    .appName("PySpark-jupyter-streaming") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:{pyspark.__version__}") \
    .config("spark.sql.streaming.checkpointLocation", "./checkpoint") \
    .getOrCreate()

:: loading settings :: url = jar:file:/Users/Rom/Documents/big%20data/wikimedia/.venv3.11/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/Rom/.ivy2.5.2/cache
The jars for the packages stored in: /Users/Rom/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-856b3e09-3312-40d6-83e1-a78fb07449a7;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.1.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.1.1 in central
	found org.apache.kafka#kafka-clients;3.9.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.8 in central
	found org.slf4j#slf4j-api;2.0.17 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.2 in central
	found org.apache.hadoop#hadoop-client-api;3.4.2 in central
	found com.google.code.findbugs#jsr

In [14]:
kafka_df = spark.readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_BROKER_URL) \
  .option("subscribe", KAFKA_TOPIC) \
  .option("startingOffsets", "earliest") \
  .load()

In [15]:
schema = StructType() \
    .add("id", IntegerType()) \
    .add("type", StringType()) \
    .add("comment", StringType()) \
    .add("user", StringType()) \
    .add("title", StringType()) \
    .add("server_name", StringType())

# Transform data to dataframe of json format
parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

In [16]:
parsed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start() 

26/01/16 13:35:38 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [17]:
parsed_df.createOrReplaceTempView("parsed_df")

spark.sql("select user, count(*) as count from parsed_df group by user") \
.writeStream \
.outputMode("complete") \
.format("console") \
.start() 

26/01/16 13:35:40 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


26/01/16 13:35:40 WARN MicroBatchExecution: Disabling AQE since AQE is not supported in stateful workloads.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+---------+----------+--------------------+------------------+--------------------+--------------------+
|       id|      type|             comment|              user|               title|         server_name|
+---------+----------+--------------------+------------------+--------------------+--------------------+
|     NULL|      NULL|                NULL|              NULL|                NULL|                NULL|
|     NULL|       log|UL HU Kiko (Band ...|PantheraLeo1359531|File:20252521530 ...|commons.wikimedia...|
|     NULL|categorize|[[:File:Youth's C...|          YiFeiBot|Category:Pages us...|commons.wikimedia...|
|342176019|      edit|Añadiendo Control...|            Aosbot|Museo Mixteco-Tlayúa|    es.wikipedia.org|
|527224657|categorize|[[:Категория:Футб...|   Fleur-de-farine|Категория:Футболь...|    ru.wikipedia.org|
|527224658|categorize|[[:Категория:Футб...|   Fleur-de-farine|К

In [18]:
spark.sql("select type, count(*) as count from parsed_df group by type") \
.writeStream \
.outputMode("complete") \
.format("console") \
.start() 

26/01/16 13:35:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


26/01/16 13:35:41 WARN MicroBatchExecution: Disabling AQE since AQE is not supported in stateful workloads.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+-----------------+-----+
|             user|count|
+-----------------+-----+
|     TurboSuperA+|    1|
|      K.davidow99|    1|
|         Makenzis|    1|
|      BhagyaMohan|    2|
|         乾燥蛞蝓|    1|
|             NULL|    2|
|Thegreatrebellion|    1|
|  Fleur-de-farine|    6|
|   Tomorrow123456|    1|
|        Icepinner|    1|
|           Aosbot|   10|
|           Idoc07|    1|
|    Johnsoniensis|    2|
|    ~2026-30843-1|    1|
|     Caddyshack01|    5|
|       Rkieferbot|    6|
|      Estopedist1|    1|
|      Everest2002|    1|
|   ~2025-41300-01|    1|
|        Globustut|    8|
+-----------------+-----+
only showing top 20 rows


-------------------------------------------
Batch: 0
-------------------------------------------
+----------+-----+
|      type|count|
+----------+-----+
|       new|    2|
|       log|    9|
|      NULL|    2|
|      edit|   35|
|categorize|   52|
+----------+-----+

